In [1]:
import pandas
import sqlalchemy
from matplotlib import pyplot
import seaborn
import sklearn
import warnings

%matplotlib inline

In [2]:
warnings.filterwarnings("ignore")

In [3]:
con = sqlalchemy.create_engine("""sqlite:///tester.db""")
data = pandas.read_sql_query("""SELECT * FROM zaif_ticker""", con )
del data["currency_pair"]

In [4]:
data.index = pandas.to_datetime(data.date)

In [5]:
data.tail(10)

,index,amount,currenty_pair,date,price,tid,trade_type
date,,,,,,,
2017-07-02 15:07:24,0,0.1000,btc_jpy,2017-07-02 15:07:24.000000,273515.0,57280755,ask
2017-07-02 15:07:27,0,0.0007,btc_jpy,2017-07-02 15:07:27.000000,273620.0,57280758,bid
2017-07-02 15:07:26,1,0.0102,btc_jpy,2017-07-02 15:07:26.000000,273540.0,57280757,ask
2017-07-02 15:07:26,2,0.0007,btc_jpy,2017-07-02 15:07:26.000000,273565.0,57280756,ask
2017-07-02 15:07:32,0,0.0793,btc_jpy,2017-07-02 15:07:32.000000,273515.0,57280763,ask
2017-07-02 15:07:31,1,0.0008,btc_jpy,2017-07-02 15:07:31.000000,273540.0,57280762,bid
2017-07-02 15:07:31,2,0.0008,btc_jpy,2017-07-02 15:07:31.000000,273535.0,57280761,ask
2017-07-02 15:07:30,3,0.0006,btc_jpy,2017-07-02 15:07:30.000000,273590.0,57280760,ask
2017-07-02 15:07:30,4,0.0100,btc_jpy,2017-07-02 15:07:30.000000,273610.0,57280759,ask


In [6]:
bid_data = data[data.trade_type == "bid"]
ask_data = data[data.trade_type == "ask"]

bid_resampled = bid_data.resample("10s").mean().dropna()
ask_resampled = ask_data.resample("10s").mean().dropna()

In [8]:
bid_resampled.head()

,index,amount,price,tid
date,,,,
2017-07-02 03:07:00,20.0,0.0068,279655.000000,57229992.0
2017-07-02 03:07:20,19.0,0.0317,279620.000000,57229993.0
2017-07-02 03:07:40,13.0,0.0051,279631.666667,57230002.0
2017-07-02 03:07:50,6.0,0.0158,279635.000000,57230010.0
2017-07-02 03:08:50,0.0,0.0183,279725.000000,57230018.0


In [9]:
ask_resampled.head()

,index,amount,price,tid
date,,,,
2017-07-02 03:07:20,18.0,0.0035,279570.0,57229997.0
2017-07-02 03:07:30,17.0,0.0035,279555.0,57229998.0
2017-07-02 03:07:40,11.5,0.0101,279552.5,57230003.5
2017-07-02 03:07:50,6.0,0.0035,279600.0,57230010.0
2017-07-02 03:08:00,4.0,0.0183,279580.0,57230012.0


In [23]:
price_data  = pandas.concat([bid_resampled["price"], ask_resampled["price"]],axis=1)
price_data.columns = ["bid", "ask"]

spread = price_data["bid"] - price_data["ask"]

spread.describe()

count    3208.000000
mean       56.193593
std        82.604521
min      -271.022727
25%         5.468254
50%        31.139423
75%        90.503205
max       915.946115
dtype: float64

In [ ]:
# dataset Creation
training_data = resampled[:"2017-07-02 06:30:00"]
test_data = resampled["2017-07-02 06:30:00":]

In [ ]:
resampled["price"].plot()

In [ ]:
test_data["price"].plot()

In [ ]:
training_data["price"].plot()

In [ ]:
rolling_diff_1M = training_data.rolling(6).mean()["price"].diff().dropna()
rolling_diff_5M = training_data.rolling(30).mean()["price"].diff().dropna()
rolling_diff_10M = training_data.rolling(60).mean()["price"].diff().dropna()

In [ ]:
rolling_diff_df = pandas.concat([rolling_diff_1M, rolling_diff_5M, rolling_diff_10M], axis=1).dropna()
rolling_diff_df.columns = ["1M", "5M", "10M"]

In [ ]:
rolling_diff_df.plot()

In [ ]:
rolling_1M = training_data.rolling(6).mean()["price"].dropna()
rolling_5M = training_data.rolling(30).mean()["price"].dropna()
rolling_10M = training_data.rolling(60).mean()["price"].dropna()

rolling_df = pandas.concat([rolling_1M, rolling_5M, rolling_10M], axis=1).dropna()
rolling_df.columns = ["1M", "5M", "10M"]

rolling_df[rolling_df.trade_type == "bid"].plot()

In [ ]:
# Detect Golden Cross / Dead Cross
def golden_detect(latest_3_df: pandas.DataFrame) -> bool:
    """
    >>> df = {'10M': {Timestamp('2017-06-30 16:35:20'): 282846.5542856449,
  Timestamp('2017-06-30 16:35:30'): 282849.31539675605,
  Timestamp('2017-06-30 16:35:40'): 282849.89873008936},
 '1M': {Timestamp('2017-06-30 16:35:20'): 282971.85019841272,
  Timestamp('2017-06-30 16:35:30'): 282930.29464285716,
  Timestamp('2017-06-30 16:35:40'): 282899.52083333331},
 '5M': {Timestamp('2017-06-30 16:35:20'): 282863.28952367074,
  Timestamp('2017-06-30 16:35:30'): 282869.81174589298,
  Timestamp('2017-06-30 16:35:40'): 282871.70222208346}}
    >>> df = pandas.DataFrame.from_dict(df)
    >>> golden_detect(df)
  """
    